<a href="https://colab.research.google.com/github/enthusiham/Master-s-thesis-/blob/main/08_Verb_ratio_%26_English_Readability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

In [ ]:
import spacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
import pickle

In [ ]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/df_main_final.csv")

In [ ]:
pd.set_option("display.max_columns", None)

In [ ]:
df.drop(['Unnamed: 0',	'_id', 'ref_index'], inplace=True, axis=1)

KeyError: ignored

In [ ]:
df_main = pd.read_csv("/content/drive/My Drive/Colab Notebooks/df_main_final.csv")

In [ ]:
df_main.drop(['Unnamed: 0'], inplace=True, axis=1)

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords


In [ ]:
import re


def preprocess(raw_text):

    # keep only words
    letters_only_text = re.sub("[^a-zA-Z]", " ", raw_text)

    # convert to lower case and split 
    words = letters_only_text.lower().split()

    # remove stopwords
    stopword_set = set(stopwords.words("english"))
    cleaned_words = list(set([w for w in words if w not in stopword_set]))

    return cleaned_words


In [ ]:
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from collections import Counter

tagged_list=[]
for index, row in df_main.iterrows():
    text = preprocess(row['abstract'].lower())
    #tokens = nltk.word_tokenize(text)
    text = nltk.Text(text)
    tagged = nltk.pos_tag(text)
    tagged_list.append(tagged)
df_main['pos_tag'] = tagged_list

In [ ]:
def VerbCounter(x):
   verbs = []
   for (word, pos) in x:
        if pos.startswith("VB"):
            verbs.append(word)
   return verbs

df_main["verbs"] = df_main["pos_tag"].apply(VerbCounter)
df_main["verb_count"] = df_main["verbs"].str.len()

In [ ]:
df_main.drop(['pos_tag',	'nouns',	'noun_count'], inplace=True, axis=1)

In [ ]:
df_main['verb_ratio'] = df_main['verb_count']/df_main['total_words']




In [ ]:
#2. Readability

def break_sentences(text):
	nlp = spacy.load('en_core_web_sm')
	doc = nlp(text)
	return list(doc.sents)

# Returns Number of Words in the text
def word_count(text):
	sentences = break_sentences(text)
	words = 0
	for sentence in sentences:
		words += len([token for token in sentence])
	return words

# Returns the number of sentences in the text
def sentence_count(text):
	sentences = break_sentences(text)
	return len(sentences)

# Returns average sentence length
def avg_sentence_length(text):
	words = word_count(text)
	sentences = sentence_count(text)
	average_sentence_length = float(words / sentences)
	return average_sentence_length

# Textstat is a python package, to calculate statistics from
# text to determine readability,
# complexity and grade level of a particular corpus.
# Package can be found at https://pypi.python.org/pypi/textstat
def syllables_count(word):
	return textstatistics().syllable_count(word)

# Returns the average number of syllables per
# word in the text
def avg_syllables_per_word(text):
	syllable = syllables_count(text)
	words = word_count(text)
	ASPW = float(syllable) / float(words)
	return ASPW

# Return total Difficult Words in a text
def difficult_words(text):
	
	nlp = spacy.load('en_core_web_sm')
	doc = nlp(text)
	# Find all words in the text
	words = []
	sentences = break_sentences(text)
	for sentence in sentences:
		words += [str(token) for token in sentence]

	# difficult words are those with syllables >= 2
	# easy_word_set is provide by Textstat as
	# a list of common words
	diff_words_set = set()
	
	for word in words:
		syllable_count = syllables_count(word)
		if word not in nlp.Defaults.stop_words and syllable_count >= 2:
			diff_words_set.add(word)

	return len(diff_words_set)

# A word is polysyllablic if it has more than 3 syllables
# this functions returns the number of all such words
# present in the text
def poly_syllable_count(text):
	count = 0
	words = []
	sentences = break_sentences(text)
	for sentence in sentences:
		words += [token for token in sentence]
	

	for word in words:
		syllable_count = syllables_count(word)
		if syllable_count >= 3:
			count += 1
	return count


In [ ]:
def flesch_reading_ease(text):
	"""
		Implements Flesch Formula:
		Reading Ease score = 206.835 - (1.015 × ASL) - (84.6 × ASW)
		Here,
		ASL = average sentence length (number of words
				divided by number of sentences)
		ASW = average word length in syllables (number of syllables
				divided by number of words)
	"""
	FRE = 206.835 - float(1.015 * avg_sentence_length(text)) -\
		float(84.6 * avg_syllables_per_word(text))
	return FRE

In [ ]:
!pip install textstat
from textstat.textstat import textstatistics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 105 kB 24.1 MB/s 
     |████████████████████████████████| 2.0 MB 59.0 MB/s 


In [ ]:
readability_score =[]
for index, row in df_main.iterrows():  
    score = flesch_reading_ease(row['abstract'])
    readability_score.append(score)
    

In [ ]:
df_main['readability'] = readability_score 


ValueError: ignored

In [ ]:
df_main['readability'].describe()